In [1]:
from k_level_pretrain import KData

data_folder = './dataset_construction/'
kg = KData(data_folder)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KG_processed.csv does not exist, processing...
Iterating over relations...


100%|██████████| 39/39 [00:14<00:00,  2.71it/s]


Categorizing values...


100%|██████████| 19/19 [00:00<00:00, 114.27it/s]


Iterating over node types...


100%|██████████| 7/7 [00:12<00:00,  1.82s/it]


Data preparation finished.


In [2]:
G = kg.G

In [3]:
G

Graph(num_nodes={'disease': 4174, 'drug': 40544, 'effect/phenotype': 991, 'gene/protein': 13092, 'molecule': 88812, 'pathway': 35764, 'value': 101},
      num_edges={('disease', 'rev_contraindication', 'molecule'): 16797, ('disease', 'rev_cooccurence_molecule_disease', 'molecule'): 110226, ('disease', 'rev_indication', 'molecule'): 4679, ('disease', 'rev_off-label use', 'molecule'): 1451, ('drug', 'rev_closematch', 'molecule'): 32619, ('drug', 'rev_to_drug', 'molecule'): 3083, ('drug', 'rev_type', 'molecule'): 4498, ('effect/phenotype', 'rev_drug_effect', 'molecule'): 52958, ('gene/protein', 'rev_cooccurence_molecule_gene/protein', 'molecule'): 107406, ('gene/protein', 'rev_drug_protein', 'molecule'): 20563, ('molecule', 'closematch', 'drug'): 32619, ('molecule', 'contraindication', 'disease'): 16797, ('molecule', 'cooccurence_molecule_disease', 'disease'): 110226, ('molecule', 'cooccurence_molecule_gene/protein', 'gene/protein'): 107406, ('molecule', 'cooccurence_molecule_molecule', '

In [16]:
len(kg.df.x_id.unique())

65454

In [10]:
kg.df.to_csv('./KGE/KG_processed.csv', index=False)

In [4]:
import json 

with open('./dataset_construction/idx_map.json', 'r') as f:
    idx_map = json.load(f)

with open('./dataset_construction/pretrain_molecules/downstream_cid_set.json', 'r') as f:
    downstream_cid_set = json.load(f)

In [15]:
import json 
import pickle
import numpy as np
import torch
from tqdm import tqdm

with open('./dataset_construction/idx_map.json') as f:
    idx_map = json.load(f)
with open('./dataset_construction/cid2emb.pkl', 'rb') as f:
    cid2emb = pickle.load(f)

molecule_idx2id = {value: key for key, value in idx_map['molecule'].items()}
cnt = 0

molecule_embedding = []
for i in tqdm(range(len(molecule_idx2id))):
    try:
        cid = int(float(molecule_idx2id[i]))
        molecule_embedding.append(cid2emb[cid]['emb'])
    
    # random embedding for molecules that are not in the pre-trained embedding
    except:
        molecule_embedding.append(np.random.rand(600))
        cnt+=1
        

molecule_embedding = torch.tensor(np.array(molecule_embedding))

100%|██████████| 88811/88811 [00:00<00:00, 278327.62it/s]


In [11]:
kg.G.ndata['h']['molecule'] = molecule_embedding

In [21]:
from torch_geometric.data import HeteroData

df = kg.df
hetero_data = HeteroData()

print("processing edges...")
for relation_type in tqdm(df['relation'].unique()):
    mask = df['relation'] == relation_type
    xtype = df[mask]['x_type'].unique()[0]
    ytype = df[mask]['y_type'].unique()[0]
    
    edge_index = torch.tensor(df[mask][['x_idx', 'y_idx']].to_numpy(), dtype=torch.long).t().contiguous()

    hetero_data[xtype, relation_type, ytype].edge_index = edge_index

print("processing nodes...")
unique_node_types = np.unique(np.append(np.unique(df.x_type.values), np.unique(df.y_type.values)))
for node_type in tqdm(unique_node_types):
    hetero_data[node_type].num_nodes = len(idx_map[node_type])


In [25]:
hetero_data['molecule'].num_nodes

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv
from torch_geometric.data import DataLoader

class RGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(RGNN, self).__init__()
        self.conv1 = RGCNConv(in_channels, hidden_channels, num_relations=len(hetero_data.edge_types))
        self.conv2 = RGCNConv(hidden_channels, out_channels, num_relations=len(hetero_data.edge_types))

    def forward(self, x, edge_index, edge_type):
        x = F.relu(self.conv1(x, edge_index, edge_type))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        return x

In [11]:
hetero_data['node_type']

{}

In [7]:
len(dataset)

1

In [3]:
# from txgnn import GodeKGNN

# gode_k = GodeKGNN(data = kg, 
#                     exp_name = 'Gode'
#                     )

In [4]:
# df_in = gode_k.df_valid[['x_idx', 'relation', 'y_idx']]
# # df_in[df_in.relation == 'cooccurence_disease_molecule']

# len(gode_k.df_train.relation.unique()), len(df_in.relation.unique())

(67, 67)

In [6]:
# from txgnn.utils import *

# # gode_k.g_valid_pos, gode_k.g_valid_neg = evaluate_graph_construct(gode_k.df_valid, gode_k.G, "fix_dst", 1, gode_k.device)
# g = gode_k.G
# df_valid = gode_k.df_valid

# out = {}
# df_in = df_valid[['x_idx', 'relation', 'y_idx']]
# for etype in g.canonical_etypes:
#     try:
#         df_temp = df_in[df_in.relation == etype[1]]
#     except:
#         print(etype[1])
#     src = torch.Tensor(df_temp.x_idx.values).to(gode_k.device).to(dtype = torch.int64)
#     dst = torch.Tensor(df_temp.y_idx.values).to(gode_k.device).to(dtype = torch.int64)
#     out.update({etype: (src, dst)})

# g = dgl.heterograph(out, num_nodes_dict={ntype: g.number_of_nodes(ntype) for ntype in g.ntypes})


# # ng = Full_Graph_NegSampler(g_valid, 1, "fix_dst", gode_k.device)
# weights = {
#                 etype: (g.in_degrees(etype=etype) > 0).float()
#                 for etype in g.canonical_etypes
#         }

In [ ]:
# gode_k.model_initialize(hidden_dim=100, 
#                         input_dim=100, 
#                         output_dim=100, 
#                       )

In [ ]:
# TxGNN.pretrain(n_epoch = 2, 
#                learning_rate = 1e-3,
#                batch_size = 1024, 
#                train_print_per_n = 20)